### JCIA hackathon

### 🧩 1. Chargement des données et du CSV

In [2]:
import os
import pandas as pd
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torchvision.models as models

# 📂 Configuration de base
data_dir = "/kaggle/input/african-plums-quality-and-defect-assessment-data/african_plums_dataset/african_plums"
csv_path = "/kaggle/input/african-plums-quality-and-defect-assessment-data/african_plums_dataset/plums_data.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📊 Data Augmentation + Normalisation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 📁 Chargement du dataset avec labels basés sur les dossiers
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# 📤 Split (80% train / 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


### 🧠 2. Fine-tuning de ResNet18

In [3]:
model = models.resnet18(pretrained=True)

# 🔧 Modification de la dernière couche pour 6 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


### ⚙️ 3. Entraînement du modèle


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/10 - Loss: 1.0102
Epoch 2/10 - Loss: 0.7045
Epoch 3/10 - Loss: 0.6174
Epoch 4/10 - Loss: 0.5642
Epoch 5/10 - Loss: 0.4812
Epoch 6/10 - Loss: 0.4559
Epoch 7/10 - Loss: 0.4180
Epoch 8/10 - Loss: 0.4006
Epoch 9/10 - Loss: 0.3594
Epoch 10/10 - Loss: 0.3469


### ✅ 4. Évaluation sur validation set

In [5]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 75.94%


### 🖼️ 5. Tester avec une image spécifique

In [6]:
from torchvision import transforms
from PIL import Image

img_path = os.path.join(data_dir, "cracked/cracked_plum_10.png")

img = Image.open(img_path).convert("RGB")
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
img_tensor = transform_test(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(img_tensor)
    _, pred_class = torch.max(output, 1)

# 📌 Mapping des indices vers noms de classes
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
print(f"✅ Prédit : {idx_to_class[pred_class.item()]}")


✅ Prédit : cracked
